### Import das dependências

In [4]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### Importar os datasets

In [5]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

consumos_path = data_path + '3-consumos-faturados-por-municipio-ultimos-10-anos.csv'
mobilidade_eletrica_path = data_path + '9-plr-mobilidade-eletrica.csv'
diagramas_carga_path = data_path + '22-diagrama-de-carga-por-instalacao.csv'
caracterizacao_contrato_ativo_path = data_path + '20-caracterizacao-pes-contrato-ativo.csv'
contadores_energia_path = data_path + '21-contadores-de-energia.csv'
ordens_servico_path = data_path + '15-ordens-de-servico.csv'


df_consumos = pd.read_csv(consumos_path, delimiter=';')
df_mobilidade_eletrica = pd.read_csv(mobilidade_eletrica_path, delimiter=';')
df_diagramas_carga = pd.read_csv(diagramas_carga_path, delimiter=';')
df_caracterizacao_contrato_ativo = pd.read_csv(caracterizacao_contrato_ativo_path, delimiter=';')
df_contadores_energia = pd.read_csv(contadores_energia_path, delimiter=';', low_memory=False)
df_ordens_servico = pd.read_csv(ordens_servico_path, delimiter=';')
